In [1]:
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from matplotlib.pyplot import imshow
import math
import numpy as np
from glob import glob

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
images = [image.load_img(p, target_size=(299, 299))
          for p in glob('/content/drive/MyDrive/m_산/*jpg') + glob('/content/drive/MyDrive/s_바다/*jpg')]
vector = np.asarray([image.img_to_array(img) for img in images])

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (109542160 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


In [4]:
base_model = InceptionV3(weights='imagenet', include_top=True)

96112376/96112376 [==============================] - 1s 0us/step


In [5]:
base_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 299, 299, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 149, 149, 32)         864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 149, 149, 32)         96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 149, 149, 32)         0         ['batch_normalizati

In [6]:
basic_model1 = Model(inputs=base_model.input, outputs=base_model.get_layer('avg_pool').output)

In [7]:
vectors = basic_model1.predict(preprocess_input(vector))

10/10 [==============================] - 93s 9s/step


In [12]:
len(vectors)

292

In [8]:
labels = [1] * 150 + [0] * 150

In [9]:
# 학습 데이터와 평가 데이터 구분
from sklearn.model_selection import train_test_split

# Ensure labels and vectors have the same length
labels = labels[:len(vectors)]  # Adjust labels to match the length of vectors

X_train, X_test, y_train, y_test = train_test_split(vectors, labels, test_size=0.2)

In [13]:
X_train.shape

(233, 2048)

In [16]:
# svm !
from sklearn import svm
from sklearn.metrics import accuracy_score, classification_report
model = svm.SVC()
model.fit(X_train, y_train)

SVC()

In [17]:
y_pred = model.predict(X_test)

In [18]:
print("accuracy : ", accuracy_score(y_test, y_pred))
print("Classfication Report:\n", classification_report(y_test, y_pred))

accuracy :  0.847457627118644
Classfication Report:
               precision    recall  f1-score   support

           0       0.87      0.84      0.86        32
           1       0.82      0.85      0.84        27

    accuracy                           0.85        59
   macro avg       0.85      0.85      0.85        59
weighted avg       0.85      0.85      0.85        59



In [21]:
# Xgboost !
import xgboost as xgb
model1 = xgb.XGBClassifier()
model1.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [22]:
y_pred_xgb = model1.predict(X_test)

In [23]:
print("accuracy : ", accuracy_score(y_test, y_pred_xgb))
print("Classfication Report:\n", classification_report(y_test, y_pred_xgb))

accuracy :  0.6610169491525424
Classfication Report:
               precision    recall  f1-score   support

           0       0.73      0.59      0.66        32
           1       0.61      0.74      0.67        27

    accuracy                           0.66        59
   macro avg       0.67      0.67      0.66        59
weighted avg       0.67      0.66      0.66        59

